# Template

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scienceplots
from matplotlib.gridspec import GridSpec
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, savgol_filter

plt.style.use('science')
plt.rcParams.update({'font.size': 14})

In [3]:
def uncert(length:float)->float:
    a,b = 0.3, 0.2 #mm
    return length/100*b + a

print(uncert(200)) #cm

0.7


## Versuch 1 Sammellinse
### Laplace Methode

In [60]:
r_0 = 190 #cm
dg = db = 0.1 #cm

measure1 = [
    ( 5.0,159.2),
    (15.0,158.9),
    (25.0,158.3),
    (35.0,157.4),
    (45.0,156.6),
    (55.0,155.7),
    (65.0,154.4),
    (75.0,152.1),
    (85.0,147.7),
    (90.0,141.2),
    (40.0,157.2)
] 
f1, gs, bs = [], [], []

for s,o in measure1:
    g = r_0 - o
    gs.append(g)

    b = o - s
    bs.append(b)

    l = 1/b + 1/g 
    f1.append(1/l)

bs = np.array(bs)
gs = np.array(gs)
f1 = np.array(f1)

k = -1.087 pm 0.011 cm^-1
d = D1 = 0.0417 pm 0.00031 cm^-1
D2 = 0.0384 pm 0.0007 cm^-1
------------------------------------
f1 = 23.99181608806376 pm 0.18043795869962043
f2 = 26.070243753324814 pm 0.45663533067920237


In [62]:
dbinv = 2*db/(bs**2)
dginv = 2*dg/(gs**2)

def ffit(x, k, d):
    return k*x + d

(k,d),pcov = curve_fit(ffit, 1/gs, 1/bs)
dk, dd = np.sqrt(np.diag(pcov))
D2 = -d/k
dD2 = D2*(abs(dd/d) + abs(dk/k))

ginv_fit = np.linspace(0,D2*1.01,1000)
binv_fit = ffit(ginv_fit, k, d)

fig, ax = plt.subplots(1,1, figsize=(6, 4))

ax.errorbar(x=1/gs, y=1/bs, xerr=dginv, yerr=dbinv, fmt='d', color='blue', 
                capsize=5, label=r'Messdaten', markersize=4)
ax.plot(ginv_fit, binv_fit, 'r--', label='linearer Fit')
ax.plot(ginv_fit, 0.04-ginv_fit, color='black', alpha=0.3)
ax.set_ylabel(r'$1/b$ / $\text{cm}^{-1}$', fontsize=18)
ax.set_xlabel(r'$1/g$ / $\text{cm}^{-1}$', fontsize=18)
ax.scatter([0, D2], [d, 0], 25, 'black', marker='o', label=r'Schnittpunkte')
ax.grid(True)
ax.legend(frameon=True, fontsize=18)

plt.tight_layout()
print(f'k = {k:.3f} pm {dk:.3f} cm^-1')
print(f'd = D1 = {d:.4f} pm {dd:.5f} cm^-1')
print(f'D2 = {D2:.4f} pm {dD2:.4f} cm^-1')
print("------------------------------------")
print(f'f1 = {1/d} pm {dd/(d**2)}')
print(f'f2 = {1/D2} pm {dD2/(D2**2)}')

In [45]:
dfinv = dbinv + dginv
df = dfinv*f1**2
print(f1)
print(df)

print(np.mean(f1), np.std(f1), np.mean(df))


[25.67221622 25.57308571 25.60975758 25.74348387 25.70648276 25.58525926
 25.46112    25.40947826 25.25914286 24.9856     25.62773333]
[0.14449251 0.14154723 0.13791574 0.1335652  0.12908556 0.12419182
 0.11852442 0.11161921 0.10377469 0.1000576  0.13165938]
25.51212362240482 0.21389455802030816 0.12513030574370196


### Bessel Verfahren

In [52]:
measure2 = [
    (20, 158.4, 49.9),
    (30, 157.6, 60.4),
    (40, 157.3, 71.3),
    (50, 156.1, 82.0),
    (60, 155.1, 93.0)
]
a_s, es = [], []

for s,o1,o2 in measure2:
    a = r_0 - s
    e = o1 - o2


    a_s.append(a)
    es.append(e)

da = de = 0.2
a_s = np.array(a_s)
es = np.array(es)

f = 0.25* (a_s**2 - es**2)/a_s
df = 0.25*(a_s**2 + es**2)/(a_s**2)*da + 0.5*es*de/a_s
print(f)
print(df)
print(np.mean(f), np.std(f), np.mean(df))

[25.18786765 25.23775    25.17333333 25.19498214 25.08382692]
[0.13419074 0.12920281 0.12376889 0.11693574 0.10917873]
25.175552009265246 0.0506284683478542 0.12265538258801896


## Zerstreulinse
### Methode 1

In [ ]:
measure3 = [
    (5, 93.7),
    (15,93.4),
    (25,93.0),
    (35,92.6),
    (45,92.2),
    (55,91.5),
    (65,90.2),
    (70,89.5),
    (30,92.8),
    (10,93.5),
]
bs, g_primes = [], []
db = dg_prime = 0.2

B_prime = 80.9
for rb, rl in measure3:
    b = rl - rb
    g_prime = rl - B_prime
    bs.append(b)
    g_primes.append(-g_prime)
    
bs = np.array(bs)
g_primes = np.array(g_primes)
finv = 1/bs + 1/g_primes
dfinv = db/(bs**2) + dg_prime/(g_primes**2)
f = 1/finv
df = dfinv*(f**2)

print(f)
print(df)
print(np.mean(f), np.std(f), np.mean(df))

[-14.95862978 -14.87101669 -14.71914132 -14.68235294 -14.85682451
 -14.93822394 -14.73962264 -15.3853211  -14.68212181 -14.83921016]
[0.27883336 0.29026414 0.3053248  0.32795079 0.36553565 0.43070616
 0.57080812 0.76459894 0.31538013 0.2837191 ]
-14.86724648887468 0.19732679320040664 0.3933121185046864
